This notebook plots the graph of temperature as function of age

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as  np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
# This generates the "eikosogram" of mortality as function of fever temperature, each bar in the plot
# represents the conditional probability of death given that temperature is in that bin

src_data_file = '../data/data.csv'
data = pd.read_csv(src_data_file, na_values='\\N')

num_bins = 5
var = 'FeverTemperature'

mint = data[var].min()
maxt = data[var].max()

bins = np.linspace(mint, maxt, num=num_bins)
bins_names = []
for i in range(0, len(bins) - 1):
    bins_names += [str(round(bins[i],1)) + "-" + str(round(bins[i + 1],1))]

data1 = data.copy()

data1['bin'] = pd.cut(data1[var], bins, labels=False)
data1 = data1.dropna(subset=['bin'])
died_temp = data1[data1['Disposition'] == 1].groupby(['bin']).count()['Disposition']
totals_temp = data1.groupby(['bin']).count()['Disposition']

fig, ax = plt.subplots()

bar_width = 0.35
opacity = 0.8
cfr_pdw = 100 * died_temp / totals_temp
index = np.arange(len(bins) - 1)
rects2 = plt.bar(index, cfr_pdw, bar_width,
                 alpha=opacity,
                 color=sns.xkcd_rgb["pale red"],
                 label='CFR')

plt.xlabel('Fever Temperature', labelpad=20)
plt.ylabel('CFR %')
plt.title('CFR as function of Fever Temperature')
plt.xticks(index + bar_width/2, bins_names, rotation=0)

fig.savefig('cfr_temp.pdf')

In [ ]:
# Looking at differences in fever temperature between males and females, 
# depending on
# 0 = Female
# 1 = Male

data = pd.read_csv(src_data_file, na_values="\\N")
data_male_died = data[(data["PatientSex"] == 1) & (data["Disposition"] == 1)]
data_male_surv = data[(data["PatientSex"] == 1) & (data["Disposition"] == 0)]
data_male_died.hist(column="FeverTemperature")
data_male_surv.hist(column="FeverTemperature")

data_fem_died = data[(data["PatientSex"] == 0) & (data["Disposition"] == 1)]
data_fem_surv = data[(data["PatientSex"] == 0) & (data["Disposition"] == 0)]
data_fem_died.hist(column="FeverTemperature")
data_fem_surv.hist(column="FeverTemperature")
print "Male died: ", data_male_died["FeverTemperature"].mean()
print "Male surv: ",data_male_surv["FeverTemperature"].mean()
(s, p) = stats.ttest_ind(data_male_died["FeverTemperature"],data_male_surv["FeverTemperature"],nan_policy='omit')
print "P-value of the difference between means for males: ", p

print "Female died: ",data_fem_died["FeverTemperature"].mean()
print "Female surv: ",data_fem_surv["FeverTemperature"].mean()
(s, p) = stats.ttest_ind(data_fem_died["FeverTemperature"],data_fem_surv["FeverTemperature"],nan_policy='omit')
print "P-value of the difference between means for females: ", p